# Day 16

## Part 1

We want to apply the FFT algorithm (Flawed Frequency Transmission) to a given signal.

It works by phases.
In each phase you apply some comuutation to the signal, then the new signal becomes the input for the next phase.

In a phase, given the signal `s` which is an ordered list of int, the computation is:
```
s[n] = abs[sum(s[i] * p[i] for i in range(len(s))) % 10
```

`p` is a pattern evolving like this:
* when calculating `s[0]`, the pattern is `[0, 1, 0, -1]`
* when calcuting `s[1]`, the pattern is `[0, 1, 1, 0, 0, -1, -1]`
* when calcuting `s[2]`, the pattern is `[0, 0, 1, 1, 1, 0, 0, 0, -1, -1, -1]`
* and so on

If the pattern is shorter than the signal, it reapeats.

In [1]:
from itertools import cycle, repeat

def _get_pattern(position):
    pattern = cycle([0, 1, 0, -1])
    
    for v in pattern:
        yield from repeat(v, position + 1)
            
def get_pattern(position):
    iterator = _get_pattern(position)
    # skip first
    next(iterator)
    return iterator

In [2]:
from itertools import islice

list(islice(get_pattern(0), 0, 20))

[1, 0, -1, 0, 1, 0, -1, 0, 1, 0, -1, 0, 1, 0, -1, 0, 1, 0, -1, 0]

In [3]:
list(islice(get_pattern(1), 0, 20))

[0, 1, 1, 0, 0, -1, -1, 0, 0, 1, 1, 0, 0, -1, -1, 0, 0, 1, 1, 0]

In [4]:
list(islice(get_pattern(4), 0, 20))

[0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, 0]

In [5]:
def apply_phase(signal):
    new_signal = []
    for i in range(len(signal)):
        new_signal.append(abs(sum(s*p for s, p in zip(signal, get_pattern(i)))) % 10)
        
    return new_signal

In [6]:
apply_phase([1,2,3,4,5,6,7,8])

[4, 8, 2, 2, 6, 1, 5, 8]

In [7]:
apply_phase([4, 8, 2, 2, 6, 1, 5, 8])

[3, 4, 0, 4, 0, 4, 3, 8]

In [8]:
apply_phase([3, 4, 0, 4, 0, 4, 3, 8])

[0, 3, 4, 1, 5, 5, 1, 8]

In [9]:
apply_phase([0, 3, 4, 1, 5, 5, 1, 8])

[0, 1, 0, 2, 9, 4, 9, 8]

In [10]:
signal = [8,0,8,7,1,2,2,4,5,8,5,9,1,4,5,4,6,6,1,9,0,8,3,2,1,8,6,4,5,5,9,5]
for _ in range(100):
    signal = apply_phase(signal)

signal[:8]

[2, 4, 1, 7, 6, 1, 7, 6]

In [11]:
%%time

signal = list(
    int(i) for i in
    "59756772370948995765943195844952640015210703313486295362653878290009098923609769261473534009395188480864325959786470084762607666312503091505466258796062230652769633818282653497853018108281567627899722548602257463608530331299936274116326038606007040084159138769832784921878333830514041948066594667152593945159170816779820264758715101494739244533095696039336070510975612190417391067896410262310835830006544632083421447385542256916141256383813360662952845638955872442636455511906111157861890394133454959320174572270568292972621253460895625862616228998147301670850340831993043617316938748361984714845874270986989103792418940945322846146634931990046966552"
)
for _ in range(100):
    signal = apply_phase(signal)
    
signal[:8]

CPU times: user 3.6 s, sys: 6.97 ms, total: 3.61 s
Wall time: 3.61 s


[6, 9, 5, 4, 9, 1, 5, 5]

## Part 2

We need to read the 8 digits after 1000 phases, and after an offset defined by the number constructed with the 7 first digits.

In [17]:
%%time

signal = list(
    int(i) for i in
    "59756772370948995765943195844952640015210703313486295362653878290009098923609769261473534009395188480864325959786470084762607666312503091505466258796062230652769633818282653497853018108281567627899722548602257463608530331299936274116326038606007040084159138769832784921878333830514041948066594667152593945159170816779820264758715101494739244533095696039336070510975612190417391067896410262310835830006544632083421447385542256916141256383813360662952845638955872442636455511906111157861890394133454959320174572270568292972621253460895625862616228998147301670850340831993043617316938748361984714845874270986989103792418940945322846146634931990046966552"
)*10000
offset = 5975677
for i in range(100):
    print("{}, ".format(i), end="")
    signal = apply_phase(signal)
    
signal[offset:offset + 8]

0, 

KeyboardInterrupt: 

In [18]:
len(signal) - offset

524323